In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import LogNorm
import seaborn as sns

# Ustawienia stylu i ziarna losowości
sns.set_style('darkgrid')
np.random.seed(1)

nb_of_samples = 30
sequence_len = 20
possible_values = np.array([0, 0.2, 0.4, 0.6, 0.8, 1.0])

random_indices = np.random.randint(0, len(possible_values), size=(nb_of_samples, sequence_len))
X_raw = possible_values[random_indices]

t = np.sum(X_raw == 0.4, axis=1)

# Przekształcenie wejścia dla liniowej sieci RNN (maska binarna dla wartości 0.4)
# Jest to niezbędne, ponieważ model liniowy nie potrafi odróżnić 0.4 od np. 0.2+0.2 bez nieliniowości
X = (X_raw == 0.4).astype(float)

print(f"Dane wejściowe kształt: {X_raw.shape}")
print(f"Przykładowa sekwencja: {X_raw[0]}")
print(f"Liczba wystąpień 0.4 (cel): {t[0]}")

def update_state(xk, sk, wx, wRec):
    return xk * wx + sk * wRec

def forward_states(X, wx, wRec):
    S = np.zeros((X.shape[0], X.shape[1] + 1))
    for k in range(0, X.shape[1]):
        S[:, k+1] = update_state(X[:, k], S[:, k], wx, wRec)
    return S

def loss(y, t): 
    return np.mean((t - y)**2)

def output_gradient(y, t):
    return 2. * (y - t)

def backward_gradient(X, S, grad_out, wRec):
    grad_over_time = np.zeros((X.shape[0], X.shape[1] + 1))
    grad_over_time[:, -1] = grad_out
    wx_grad = 0
    wRec_grad = 0
    for k in range(X.shape[1], 0, -1):
        wx_grad += np.sum(np.mean(grad_over_time[:, k] * X[:, k-1], axis=0))
        wRec_grad += np.sum(np.mean(grad_over_time[:, k] * S[:, k-1]), axis=0)
        grad_over_time[:, k-1] = grad_over_time[:, k] * wRec
    return (wx_grad, wRec_grad), grad_over_time

def update_rprop(X, t, W, W_prev_sign, W_delta, eta_p, eta_n):
    S = forward_states(X, W[0], W[1])
    grad_out = output_gradient(S[:, -1], t)
    W_grads, _ = backward_gradient(X, S, grad_out, W[1])
    W_sign = np.sign(W_grads)
    
    for i in range(len(W)):
        if W_sign[i] == W_prev_sign[i]:
            W_delta[i] *= eta_p
        else:
            W_delta[i] *= eta_n
    return W_delta, W_sign

eta_p, eta_n = 1.2, 0.5
W = [-1.5, 2.0]  
W_delta = [0.001, 0.001]
W_sign = [0, 0]

for i in range(500):
    W_delta, W_sign = update_rprop(X, t, W, W_sign, W_delta, eta_p, eta_n)
    for j in range(len(W)):
        W[j] -= W_sign[j] * W_delta[j]

print(f"\nFinalne wagi: wx = {W[0]:.4f}, wRec = {W[1]:.4f}")

test_idx = 0
y_final = forward_states(X[test_idx:test_idx+1], W[0], W[1])[:, -1]
print(f"Test dla sekwencji nr {test_idx}:")
print(f"  Cel: {t[test_idx]}, Wyjście modelu: {y_final[0]:.2f}")

final_loss = loss(forward_states(X, W[0], W[1])[:, -1], t)
print(f"Końcowa strata (MSE): {final_loss:.6f}")

Dane wejściowe (X_raw) kształt: (30, 20)
Przykładowa sekwencja: [1.  0.6 0.8 0.  0.2 0.6 1.  0.  0.  0.2 0.8 1.  0.8 0.2 0.4 0.8 1.  0.4
 0.8 0.6]
Liczba wystąpień 0.4 (cel): 2

Finalne wagi: wx = 1.0000, wRec = 1.0000
Test dla sekwencji nr 0:
  Cel: 2, Wyjście modelu: 2.00
Końcowa strata (MSE): 0.000000
